# Optim Bottom-Up

## Setup


In [ ]:
%load_ext autoreload
%autoreload 2
from importlib import reload
import logging

import torch

import tensorcraft as tc

tc.set_logger_config(level = logging.INFO)

ALPHA = 1e-6 # 1 micro second of latency (Maybe bigger)
BETA=64.0/( 200.0 * 1e9) # 200 GBits per second bandwidth

NODE_LIMIT = 1000
TOP_K = 15
PATH_COST_W = 1.02
ESTIMATE_W = 1.00

In [ ]:
columns = ["Step #", "Operation", "Distribution", "Cost[s]", "Memory Usage [MB]"]
columns_width = [8, 20, 40, 8, 8]

type_size = 8

def print_path(path: list[tuple[str, any, float]], tensor_shape: torch.Size):

    line = " & ".join(f"{col:<{width}}" for col, width in zip(columns, columns_width)) + " \\\\"
    print(line)
    for i, (op, s_dist, s_cost) in enumerate(path):
        line = f"{i:<{columns_width[0]}} & {op:<{columns_width[1]}} & {s_dist.latexStr():<{columns_width[2]}} & {s_cost:<{columns_width[3]}} & {s_dist.maxNumElements(tensor_shape) * 8 / 10**6:<{columns_width[4]}} \\\\" 
        print(line)


In [ ]:

def mem_constrained_filter(shape: torch.Size, start_dist: tc.dist.MultiAxisDist, target_dist: tc.dist.MultiAxisDist, current_dist: tc.dist.MultiAxisDist ) -> bool:
    max_n_elements = max(start_dist.maxNumElements(shape), target_dist.maxNumElements(shape))
    return max_n_elements < current_dist.maxNumElements(shape)

## Redistributors

Given a tensor shape, a starting distribution and a target distribution, creates a sequence of collective ops to reach the target dist while optimizing for different metrics.

### Problem 1 ( Tiled Matrix to Row cyclic)

Shifting from a tiled matrix, to a row cyclic distribution

In [ ]:
tensor_shape = torch.Size([100000, 100000])
mesh = torch.Size([2,4])
dist = tc.dist.MultiAxisDist(mesh, ((0,), (1,),), 100) 
target_dist = tc.dist.MultiAxisDist(mesh, ((0,1), None), 1)

In [ ]:
%%time
naive_rdist = tc.optim.NaiveGathererRedist(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA)

sequence, total_cost = naive_rdist.redistribute(tensor_shape, dist, target_dist)
print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.2f}s")


Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,1 \}(100,100)}$            & 0        & 10000.0  \\
1        & allgather_*          & $T_{\perp\{ \emptyset,\emptyset \}(\emptyset,\emptyset)}$ & 2.800003 & 80000.0  \\
2        & split_*              & $T_{\perp\{ (0,1),\emptyset \}(1,\emptyset)}$ & 0        & 10000.0  \\
Total cost: 2.80s
CPU times: user 662 ms, sys: 48.5 ms, total: 710 ms
Wall time: 624 ms


In [ ]:
%%time
mem_constrained_dist = tc.optim.AStarRedistributor(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA, node_filter=mem_constrained_filter, path_cost_w=PATH_COST_W, estimate_w=ESTIMATE_W, top_k=TOP_K, node_limit=1000, max_depth=6)
sequence, total_cost = mem_constrained_dist.redistribute(tensor_shape, dist, target_dist)

print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.3f}s")

[2025-05-02 09:51:01,334][tensorcraft.util.route_finder][find_routes][INFO] - Explored 38 nodes, found 14 possible paths.
Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,1 \}(100,100)}$            & 0        & 10000.0  \\
1        & alltoall_0_1_-1      & $T_{\perp\{ \emptyset,(0,1) \}(\emptyset,100)}$ & 0.40000099999999994 & 10000.0  \\
2        & alltoall_minor_1_0_1 & $T_{\perp\{ 1,0 \}(1,400)}$              & 1.200002 & 10000.0  \\
3        & alltoall_1_0_-1      & $T_{\perp\{ (0,1),\emptyset \}(1,\emptyset)}$ & 0.40000099999999994 & 10000.0  \\
Total cost: 2.000s
CPU times: user 2min 26s, sys: 3.51 s, total: 2min 30s
Wall time: 1min 46s


In [ ]:
%%time
astar_redist = tc.optim.AStarRedistributor(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA, path_cost_w=PATH_COST_W, estimate_w=ESTIMATE_W, top_k=TOP_K, node_limit=1000, max_depth=6)
sequence, total_cost = astar_redist.redistribute(tensor_shape, dist, target_dist)

print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.3f}s")

[2025-05-02 09:51:52,600][tensorcraft.util.route_finder][find_routes][INFO] - Explored 41 nodes, found 15 possible paths.
Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,1 \}(100,100)}$            & 0        & 10000.0  \\
1        & allgather_1          & $T_{\perp\{ 0,\emptyset \}(100,\emptyset)}$ & 1.200002 & 40000.0  \\
2        & split_minor_0_1_1    & $T_{\perp\{ (0,1),\emptyset \}(25,\emptyset)}$ & 0.0      & 10000.0  \\
3        & changeBlockSize_0_1  & $T_{\perp\{ (0,1),\emptyset \}(1,\emptyset)}$ & 0.35000299999999995 & 10000.0  \\
Total cost: 1.550s
CPU times: user 1min 14s, sys: 1.2 s, total: 1min 15s
Wall time: 51.1 s


## Problem 2

In [ ]:
tensor_shape = torch.Size([500, 100, 100, 8])
dist = tc.dist.MultiAxisDist(torch.Size([4, 4, 2]), ((0,), None, None, (1, 2)), 1)
target_dist = tc.dist.MultiAxisDist(
    torch.Size([4, 4, 2]), ((0,), (1,), (2,), None), (1, 25, 25, None)
)

In [ ]:
%%time
naive_rdist = tc.optim.NaiveGathererRedist(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA)

sequence, total_cost = naive_rdist.redistribute(tensor_shape, dist, target_dist)
print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.2f}s")

Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,\emptyset,\emptyset,(1,2) \}(1,\emptyset,\emptyset,1)}$ & 0        & 10.0     \\
1        & allgather_*          & $T_{\perp\{ \emptyset,\emptyset,\emptyset,\emptyset \}(\emptyset,\emptyset,\emptyset,\emptyset)}$ & 0.012405 & 320.0    \\
2        & split_*              & $T_{\perp\{ 0,1,2,\emptyset \}(1,25,25,\emptyset)}$ & 0        & 10.0     \\
Total cost: 0.01s
CPU times: user 5.52 ms, sys: 3.28 ms, total: 8.8 ms
Wall time: 9.01 ms


In [ ]:
%%time
mem_constrained_dist = tc.optim.AStarRedistributor(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA, node_filter=mem_constrained_filter, path_cost_w=PATH_COST_W, estimate_w=ESTIMATE_W, top_k=TOP_K, node_limit=1000, max_depth=6)
sequence, total_cost = mem_constrained_dist.redistribute(tensor_shape, dist, target_dist)

print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.3f}s")

[2025-05-02 09:52:05,593][tensorcraft.util.route_finder][find_routes][INFO] - Explored 214 nodes, found 5 possible paths.
Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,\emptyset,\emptyset,(1,2) \}(1,\emptyset,\emptyset,1)}$ & 0        & 10.0     \\
1        & alltoall_minor_3_2_25 & $T_{\perp\{ 0,\emptyset,2,1 \}(1,\emptyset,25,2)}$ & 0.000401 & 10.0     \\
2        & alltoall_3_1_25      & $T_{\perp\{ 0,1,2,\emptyset \}(1,25,25,\emptyset)}$ & 0.001202 & 10.0     \\
Total cost: 0.002s
CPU times: user 12.8 s, sys: 366 μs, total: 12.8 s
Wall time: 12.8 s


In [ ]:
%%time
astar_redist = tc.optim.AStarRedistributor(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA, path_cost_w=PATH_COST_W, estimate_w=ESTIMATE_W, top_k=TOP_K, node_limit=1000, max_depth=6)
sequence, total_cost = astar_redist.redistribute(tensor_shape, dist, target_dist)

print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.3f}s")

[2025-05-02 09:52:47,513][tensorcraft.util.route_finder][find_routes][INFO] - Explored 1000 nodes, found 9 possible paths.
Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,\emptyset,\emptyset,(1,2) \}(1,\emptyset,\emptyset,1)}$ & 0        & 10.0     \\
1        & alltoall_minor_3_2_25 & $T_{\perp\{ 0,\emptyset,2,1 \}(1,\emptyset,25,2)}$ & 0.000401 & 10.0     \\
2        & allgather_1          & $T_{\perp\{ 0,\emptyset,2,\emptyset \}(1,\emptyset,25,\emptyset)}$ & 0.001202 & 40.0     \\
3        & split_1_1_25         & $T_{\perp\{ 0,1,2,\emptyset \}(1,25,25,\emptyset)}$ & 0.0      & 10.0     \\
Total cost: 0.002s
CPU times: user 41.9 s, sys: 7.58 ms, total: 41.9 s
Wall time: 41.9 s


## Problem 3

In [ ]:
tensor_shape = torch.Size([1000, 1000, 1000])
dist = tc.dist.MultiAxisDist(torch.Size([2, 2, 2]), ((0,), (1,), (2,)), 1)
target_dist = tc.dist.MultiAxisDist(
    torch.Size([2, 2, 2]), ((0, 1), (2,), None), (1, 50, None)
)

In [ ]:
%%time
naive_rdist = tc.optim.NaiveGathererRedist(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA)

sequence, total_cost = naive_rdist.redistribute(tensor_shape, dist, target_dist)
print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.2f}s")

Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,1,2 \}(1,1,1)}$            & 0        & 1000.0   \\
1        & allgather_*          & $T_{\perp\{ \emptyset,\emptyset,\emptyset \}(\emptyset,\emptyset,\emptyset)}$ & 0.28000299999999995 & 8000.0   \\
2        & split_*              & $T_{\perp\{ (0,1),2,\emptyset \}(1,50,\emptyset)}$ & 0        & 1000.0   \\
Total cost: 0.28s
CPU times: user 33 ms, sys: 0 ns, total: 33 ms
Wall time: 35.6 ms


In [ ]:
%%time
mem_constrained_dist = tc.optim.AStarRedistributor(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA, node_filter=mem_constrained_filter, path_cost_w=PATH_COST_W, estimate_w=ESTIMATE_W, top_k=TOP_K, node_limit=1000, max_depth=6)
sequence, total_cost = mem_constrained_dist.redistribute(tensor_shape, dist, target_dist)

print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.3f}s")

[2025-05-02 09:56:14,786][tensorcraft.util.route_finder][find_routes][INFO] - Explored 1000 nodes, found 18 possible paths.
Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,1,2 \}(1,1,1)}$            & 0        & 1000.0   \\
1        & alltoall_1_0_-1      & $T_{\perp\{ (1,0),\emptyset,2 \}(1,\emptyset,1)}$ & 0.040001 & 1000.0   \\
2        & permute_1_0          & $T_{\perp\{ (0,1),\emptyset,2 \}(1,\emptyset,1)}$ & 0.080001 & 1000.0   \\
3        & alltoall_2_1_50      & $T_{\perp\{ (0,1),2,\emptyset \}(1,50,\emptyset)}$ & 0.040001 & 1000.0   \\
Total cost: 0.160s
CPU times: user 3min 26s, sys: 54.8 ms, total: 3min 26s
Wall time: 3min 27s


In [ ]:
%%time
astar_redist = tc.optim.AStarRedistributor(tc.optim.IdealLowerBoundsCM(), alpha=ALPHA, beta=BETA, path_cost_w=PATH_COST_W, estimate_w=ESTIMATE_W, top_k=TOP_K, node_limit=1000, max_depth=6)
sequence, total_cost = astar_redist.redistribute(tensor_shape, dist, target_dist)

print_path(sequence, tensor_shape)
print(f"Total cost: {total_cost:.3f}s")

[2025-05-02 09:57:29,049][tensorcraft.util.route_finder][find_routes][INFO] - Explored 1000 nodes, found 24 possible paths.
Step #   & Operation            & Distribution                             & Cost[s]  & Memory Usage [MB] \\
0        &                      & $T_{\perp\{ 0,1,2 \}(1,1,1)}$            & 0        & 1000.0   \\
1        & alltoall_1_0_-1      & $T_{\perp\{ (1,0),\emptyset,2 \}(1,\emptyset,1)}$ & 0.040001 & 1000.0   \\
2        & allgather_0          & $T_{\perp\{ 1,\emptyset,2 \}(2,\emptyset,1)}$ & 0.040001 & 2000.0   \\
3        & split_0_0_1          & $T_{\perp\{ (0,1),\emptyset,2 \}(2,\emptyset,1)}$ & 0.0      & 1000.0   \\
4        & changeBlockSize_0_1  & $T_{\perp\{ (0,1),\emptyset,2 \}(1,\emptyset,1)}$ & 0.030001999999999997 & 1000.0   \\
5        & allgather_2          & $T_{\perp\{ (0,1),\emptyset,\emptyset \}(1,\emptyset,\emptyset)}$ & 0.040001 & 2000.0   \\
6        & split_1_2_50         & $T_{\perp\{ (0,1),2,\emptyset \}(1,50,\emptyset)}$ & 0.0      & 